In [48]:
import pickle
import copy
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
import time
import requests
from collections import defaultdict
from pymongo import MongoClient

In [2]:
client = MongoClient()
db = client.boxscores

In [96]:
client.list_database_names()
CHO = pd.DataFrame(list(db['CHO'].find()))


In [99]:
CHO[CHO.year=='2018']

,_id,content,url,year
246,5b66be49f227df0bf9c11c29,"b'\n<!DOCTYPE html>\n<html data-version=""kleck...",https://www.basketball-reference.com/boxscores...,2018
247,5b66be4df227df0bf9c11c2a,"b'\n<!DOCTYPE html>\n<html data-version=""kleck...",https://www.basketball-reference.com/boxscores...,2018
248,5b66be50f227df0bf9c11c2b,"b'\n<!DOCTYPE html>\n<html data-version=""kleck...",https://www.basketball-reference.com/boxscores...,2018
249,5b66be53f227df0bf9c11c2c,"b'\n<!DOCTYPE html>\n<html data-version=""kleck...",https://www.basketball-reference.com/boxscores...,2018
250,5b66be57f227df0bf9c11c2d,"b'\n<!DOCTYPE html>\n<html data-version=""kleck...",https://www.basketball-reference.com/boxscores...,2018
251,5b66be5af227df0bf9c11c2e,"b'\n<!DOCTYPE html>\n<html data-version=""kleck...",https://www.basketball-reference.com/boxscores...,2018
252,5b66be5df227df0bf9c11c2f,"b'\n<!DOCTYPE html>\n<html data-version=""kleck...",https://www.basketball-reference.com/boxscores...,2018
253,5b66be61f227df0bf9c11c30,"b'\n<!DOCTYPE html>\n<html data-version=""kleck...",https://www.basketball-reference.com/boxscores...,2018
254,5b66be64f227df0bf9c11c31,"b'\n<!DOCTYPE html>\n<html data-version=""kleck...",https://www.basketball-reference.com/boxscores...,2018
255,5b66be67f227df0bf9c11c32,"b'\n<!DOCTYPE html>\n<html data-version=""kleck...",https://www.basketball-reference.com/boxscores...,2018


In [65]:
def box_score_url_creator(team:str,year:str,baseurl:str)->list:
    return [baseurl + '/teams/' + team + '/' + year + '_games.html']

def url_list_generator(teams:list,years:list):
    biglist = {}
    baseurl = 'https://www.basketball-reference.com'
    for team in teams:
        biglist[team] = {}
        biglist[team]['year'] = {}
        for year in years:
            biglist[team]['year'][int(year)] = box_score_url_creator(team,year,baseurl)
    return biglist

def get_box_score_url(soup):
    container = []
    baseurl = 'https://www.basketball-reference.com'

    for link in soup.find_all('a'):
        k = str(link.get('href'))
        if k.startswith('/boxscores/20'):
            container.append(baseurl + k)
    return container[:82]
#this code not working, look debug
def soup_maker(dct:dict):
    boxscores = {}
    for team in dct.keys():
        boxscores[team] = {}
        for year in dct[team]['year'].keys():
            boxscores[team]['year'] = {}
            url = dct[team]['year'][int(year)][0]
            r = requests.get(url)
            soup = BeautifulSoup(r.content,'html.parser')
            boxscores[team]['year'][int(year)] = get_box_score_url(soup)
            time.sleep(3)
    return boxscores
def mongo_populator(teamlist,years,dct):
    for team in teamlist:
        if team == 'CHA':
            collection = db['CHO']
            team = 'CHO'
        else:
            collection = db[f'{team}']
        for year in years:
            for items in dct['CHA']['year'][int(year)]:
                r = requests.get(items)
                time.sleep(3)
                boxscore = {'year': year,
                            'url':items,
                            'content': r.content }
                db['CHO'].insert_one(boxscore)

In [92]:
dct = url_list_generator(['CHA'],['2008'])

In [93]:
dik = soup_maker(dct)


In [94]:
dik

{'CHA': {'year': {2008: ['https://www.basketball-reference.com/boxscores/200711020CHA.html',
    'https://www.basketball-reference.com/boxscores/200711040MIA.html',
    'https://www.basketball-reference.com/boxscores/200711060CHA.html',
    'https://www.basketball-reference.com/boxscores/200711070PHI.html',
    'https://www.basketball-reference.com/boxscores/200711090CHA.html',
    'https://www.basketball-reference.com/boxscores/200711110CHA.html',
    'https://www.basketball-reference.com/boxscores/200711130CHA.html',
    'https://www.basketball-reference.com/boxscores/200711140ATL.html',
    'https://www.basketball-reference.com/boxscores/200711170CHA.html',
    'https://www.basketball-reference.com/boxscores/200711190CHA.html',
    'https://www.basketball-reference.com/boxscores/200711210CHA.html',
    'https://www.basketball-reference.com/boxscores/200711230ORL.html',
    'https://www.basketball-reference.com/boxscores/200711240CHA.html',
    'https://www.basketball-reference.com/b

In [95]:
#mongo_populator(['CHA'],['2008'],dik)

In [23]:
#import pymongo
#from pymongo import MongoClient
#client = MongoClient()
#scrape CHA 2014 data, put into CHO database?
#db = client.boxscores <---


In [6]:
boxscore = pd.read_pickle('final_df.pkl')

In [4]:
spread = pd.read_pickle('spread_df.pkl')

In [183]:
teams = ['ATL','BOS','BRK','CHI','CHO','CLE','DAL','DEN','DET','GSW','HOU','IND','LAC','LAL','MEM','MIA','MIL','MIN','NOP','NYK','OKC','ORL', \
'PHI','PHO','POR','SAC','SAS','TOR','UTA','WAS']
years = ['2014','2015','2016','2017','2018']
team_dic ={'Dallas':'DAL','Boston':'BOS','Toronto':'TOR','Denver':'DEN','Philadelphia':'PHI','New York':'NYK','Orlando':'ORL','Cleveland':'CLE','Detroit':'DET'\
           ,'Miami':'MIA','Charlotte':'CHO','Houston':'HOU','San Antonio':'SAS','LA Clippers':'LAC','Washington':'WAS'\
          , 'Oklahoma City':'OKC', 'Milwaukee':'MIL','Phoenix':'PHO','Sacramento':'SAC','New Orleans':'NOP'\
          , 'Indiana':'IND','Portland':'POR','Brooklyn':'BRK', 'Golden State':'GSW','Chicago':'CHI'\
          , 'LA Lakers':'LAL','Memphis':'MEM','Atlanta':'ATL','Utah':'UTA','Minnesota':'MIN'}

In [49]:
len(team_dic), team_dic

(30,
 {'Dallas': 'DAL',
  'Boston': 'BOS',
  'Toronto': 'TOR',
  'Denver': 'DEN',
  'Philadelphia': 'PHI',
  'New York': 'NYK',
  'Orlando': 'ORL',
  'Cleveland': 'CLE',
  'Detroit': 'DET',
  'Miami': 'MIA',
  'Charlotte': 'CHO',
  'Houston': 'HOU',
  'San Antonio': 'SAS',
  'LA Clippers': 'LAC',
  'Washington': 'WAS',
  'Oklahoma City': 'OKC',
  'Milwaukee': 'MIL',
  'Phoenix': 'PHO',
  'Sacramento': 'SAC',
  'New Orleans': 'NOP',
  'Indiana': 'IND',
  'Portland': 'POR',
  'Brooklyn': 'BRK',
  'Golden State': 'GSW',
  'Chicago': 'CHI',
  'LA Lakers': 'LAL',
  'Memphis': 'MEM',
  'Atlanta': 'ATL',
  'Utah': 'UTA',
  'Minnesota': 'MIN'})

In [53]:
def switch_to_key(x):
    for k,v in team_dic.items():
        if k in x:
            return v

In [55]:
opponents = spread.opp.apply(switch_to_key).values

In [56]:
spread.opp = opponents

In [58]:
spread.to_pickle('spread_df.pkl')

In [22]:
print('hou')

hou


In [23]:
boxscore.columns[0] = 'team' 
#boxscore.columns[1] = 'year'

NameError: name 'boxscore' is not defined

In [175]:
columns = boxscore.columns.values
columns[0] = 'team' 
columns[1] = 'year'
boxscore.columns = columns
boxscore = copy.deepcopy(boxscore)

In [189]:
import copy
def dataframe_separator(df,teams:list):
    lst = []
    for team in teams:
        lst.append(copy.deepcopy(df[df.team==f'{team}']))
    return dict(zip(teams,lst))

In [78]:
box_2014_df = copy.deepcopy(boxscore[boxscore.year=='2014'])

In [180]:
dataframe_separator(boxscore,teams)
big_dic = _

In [182]:
big_dic['ATL']

,team,year,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,opp_ft_pct,opp_orb,opp_drb,opp_trb,opp_ast,opp_stl,opp_blk,opp_tov,opp_pf,opp_pts
0,ATL,2008,240.0,36.0,79.0,0.456,4.0,15.0,0.267,25.0,...,0.800,7.0,31.0,38.0,14.0,4.0,10.0,10.0,25.0,94.0
1,ATL,2008,240.0,33.0,72.0,0.458,4.0,10.0,0.400,21.0,...,0.667,13.0,24.0,37.0,26.0,10.0,10.0,14.0,20.0,92.0
2,ATL,2008,240.0,26.0,68.0,0.382,7.0,18.0,0.389,23.0,...,0.667,13.0,24.0,37.0,26.0,10.0,10.0,14.0,20.0,92.0
3,ATL,2008,240.0,41.0,90.0,0.456,4.0,16.0,0.250,19.0,...,0.882,10.0,30.0,40.0,21.0,5.0,3.0,14.0,18.0,96.0
4,ATL,2008,240.0,30.0,70.0,0.429,5.0,11.0,0.455,18.0,...,0.720,8.0,31.0,39.0,30.0,8.0,6.0,18.0,20.0,106.0
5,ATL,2008,240.0,30.0,77.0,0.390,3.0,20.0,0.150,27.0,...,0.800,18.0,35.0,53.0,26.0,4.0,3.0,12.0,25.0,101.0
6,ATL,2008,240.0,46.0,77.0,0.597,3.0,6.0,0.500,22.0,...,0.800,18.0,35.0,53.0,26.0,4.0,3.0,12.0,25.0,101.0
7,ATL,2008,290.0,45.0,99.0,0.455,6.0,23.0,0.261,27.0,...,0.800,18.0,35.0,53.0,26.0,4.0,3.0,12.0,25.0,101.0
8,ATL,2008,240.0,40.0,84.0,0.476,2.0,13.0,0.154,14.0,...,0.625,13.0,31.0,44.0,27.0,10.0,7.0,14.0,23.0,105.0
9,ATL,2008,240.0,33.0,79.0,0.418,0.0,8.0,0.000,17.0,...,0.800,12.0,27.0,39.0,22.0,4.0,4.0,15.0,17.0,95.0


In [80]:
box_2014_df.index = range(1,len(box_2014_df)+1)

In [311]:
box_2014_df[box_2014_df.team=='CHO']


,team,year,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,opp_ft_pct,opp_orb,opp_drb,opp_trb,opp_ast,opp_stl,opp_blk,opp_tov,opp_pf,opp_pts


In [168]:
sorted(teams)
teams = sorted(teams)
teams

['ATL',
 'BOS',
 'BRK',
 'CHI',
 'CHO',
 'CLE',
 'DAL',
 'DEN',
 'DET',
 'GSW',
 'HOU',
 'IND',
 'LAC',
 'LAL',
 'MEM',
 'MIA',
 'MIL',
 'MIN',
 'NOP',
 'NYK',
 'OKC',
 'ORL',
 'PHI',
 'PHO',
 'POR',
 'SAC',
 'SAS',
 'TOR',
 'UTA',
 'WAS']

In [164]:
#atl_2014 = copy.deepcopy(box_2014_df[box_2014_df.team=='ATL'])
bos_2014 = copy.deepcopy(box_2014_df[box_2014_df.team=='BOS'])
brk_2014 = copy.deepcopy(box_2014_df[box_2014_df.team=='BRK'])
chi_2014 = copy.deepcopy(box_2014_df[box_2014_df.team=='CHI'])

SyntaxError: invalid syntax (<ipython-input-164-e5406bada38f>, line 1)

In [169]:
lst = []
for team in teams:
    lst.append(copy.deepcopy(box_2014_df)[box_2014_df.team==f'{team}'])


In [174]:
d = dict(zip(teams,lst))
d['ATL']

,team,year,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,opp_ft_pct,opp_orb,opp_drb,opp_trb,opp_ast,opp_stl,opp_blk,opp_tov,opp_pf,opp_pts
1,ATL,2014,240.0,37.0,76.0,0.487,8.0,24.0,0.333,27.0,...,0.792,7.0,35.0,42.0,31.0,7.0,2.0,20.0,22.0,118.0
2,ATL,2014,240.0,36.0,77.0,0.468,10.0,23.0,0.435,20.0,...,0.571,10.0,32.0,42.0,15.0,5.0,6.0,12.0,25.0,95.0
3,ATL,2014,240.0,40.0,90.0,0.444,9.0,22.0,0.409,14.0,...,0.833,6.0,47.0,53.0,27.0,5.0,7.0,13.0,24.0,105.0
4,ATL,2014,240.0,39.0,81.0,0.481,8.0,22.0,0.364,19.0,...,0.952,10.0,29.0,39.0,22.0,10.0,4.0,18.0,26.0,100.0
5,ATL,2014,240.0,43.0,94.0,0.457,8.0,26.0,0.308,13.0,...,0.647,10.0,35.0,45.0,28.0,4.0,8.0,16.0,18.0,109.0
6,ATL,2014,240.0,43.0,90.0,0.478,6.0,19.0,0.316,12.0,...,0.556,9.0,27.0,36.0,22.0,9.0,2.0,17.0,24.0,94.0
7,ATL,2014,240.0,35.0,76.0,0.461,10.0,27.0,0.370,23.0,...,0.556,9.0,27.0,36.0,22.0,9.0,2.0,17.0,24.0,94.0
8,ATL,2014,240.0,35.0,77.0,0.455,5.0,24.0,0.208,16.0,...,0.692,8.0,29.0,37.0,24.0,11.0,3.0,3.0,19.0,95.0
9,ATL,2014,240.0,39.0,81.0,0.481,10.0,23.0,0.435,25.0,...,0.783,9.0,30.0,39.0,21.0,6.0,4.0,16.0,19.0,103.0
10,ATL,2014,240.0,44.0,78.0,0.564,9.0,22.0,0.409,13.0,...,0.750,19.0,29.0,48.0,19.0,6.0,3.0,14.0,20.0,90.0


In [96]:
atl_stats = atl_2014.describe().columns

In [111]:
atl_2014[atl_stats]

,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,...,opp_ft_pct,opp_orb,opp_drb,opp_trb,opp_ast,opp_stl,opp_blk,opp_tov,opp_pf,opp_pts
1,240.0,37.0,76.0,0.487,8.0,24.0,0.333,27.0,35.0,0.771,...,0.792,7.0,35.0,42.0,31.0,7.0,2.0,20.0,22.0,118.0
2,240.0,36.0,77.0,0.468,10.0,23.0,0.435,20.0,31.0,0.645,...,0.571,10.0,32.0,42.0,15.0,5.0,6.0,12.0,25.0,95.0
3,240.0,40.0,90.0,0.444,9.0,22.0,0.409,14.0,26.0,0.538,...,0.833,6.0,47.0,53.0,27.0,5.0,7.0,13.0,24.0,105.0
4,240.0,39.0,81.0,0.481,8.0,22.0,0.364,19.0,26.0,0.731,...,0.952,10.0,29.0,39.0,22.0,10.0,4.0,18.0,26.0,100.0
5,240.0,43.0,94.0,0.457,8.0,26.0,0.308,13.0,19.0,0.684,...,0.647,10.0,35.0,45.0,28.0,4.0,8.0,16.0,18.0,109.0
6,240.0,43.0,90.0,0.478,6.0,19.0,0.316,12.0,17.0,0.706,...,0.556,9.0,27.0,36.0,22.0,9.0,2.0,17.0,24.0,94.0
7,240.0,35.0,76.0,0.461,10.0,27.0,0.370,23.0,28.0,0.821,...,0.556,9.0,27.0,36.0,22.0,9.0,2.0,17.0,24.0,94.0
8,240.0,35.0,77.0,0.455,5.0,24.0,0.208,16.0,21.0,0.762,...,0.692,8.0,29.0,37.0,24.0,11.0,3.0,3.0,19.0,95.0
9,240.0,39.0,81.0,0.481,10.0,23.0,0.435,25.0,31.0,0.806,...,0.783,9.0,30.0,39.0,21.0,6.0,4.0,16.0,19.0,103.0
10,240.0,44.0,78.0,0.564,9.0,22.0,0.409,13.0,17.0,0.765,...,0.750,19.0,29.0,48.0,19.0,6.0,3.0,14.0,20.0,90.0


In [91]:
atl_2014_avg = (atl_2014[atl_stats].mean(),atl_2014.std())

In [93]:
atl_2014_avg = tuple(zip(atl_2014_avg[0],atl_2014_avg[1]))

In [101]:
atl_2014_dict = dict(zip(atl_stats,atl_2014_avg))

In [102]:
atl_2014_dict

{'mp': (241.82926829268294, 7.6381189560250835),
 'fg': (37.329268292682926, 4.993959012495839),
 'fga': (81.5609756097561, 7.109544204062677),
 'fg_pct': (0.45852439024390246, 0.05410237794225243),
 'fg3': (9.365853658536585, 3.342624809980189),
 'fg3a': (25.804878048780488, 5.466935459494435),
 'fg3_pct': (0.3597926829268291, 0.10011336016771717),
 'ft': (16.975609756097562, 5.702448271600015),
 'fta': (21.73170731707317, 6.836808114700145),
 'ft_pct': (0.7829634146341459, 0.09784170197306692),
 'orb': (8.695121951219512, 3.1921548292775666),
 'drb': (31.28048780487805, 4.563068033961963),
 'trb': (39.97560975609756, 5.528770116972073),
 'ast': (24.890243902439025, 4.714061176871101),
 'stl': (8.292682926829269, 3.057021082397605),
 'blk': (3.975609756097561, 2.1771859159236344),
 'tov': (14.5, 3.8880951570939315),
 'pf': (19.23170731707317, 4.010168496468602),
 'pts': (101.0, 11.953614051360738),
 'opp_mp': (242.1341463414634, 8.959114532859115),
 'opp_fg': (38.73170731707317, 4.402

In [142]:
new_k = pd.concat([k,pd.DataFrame([atl_2014[atl_stats].mean(),atl_2014[atl_stats].std()])],copy=True)

In [116]:
#for any matchup i need to generate a hypothetical game with some feature engineering
#dataframe of predicted spread + std(ortg - ortg, var)
#x will look like predicted spread, some feature engineering, actual spread, trained to outcome
#first try to fit data to spread?

SyntaxError: invalid syntax (<ipython-input-116-a35077a93b18>, line 1)

In [141]:
k

,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,...,opp_ft_pct,opp_orb,opp_drb,opp_trb,opp_ast,opp_stl,opp_blk,opp_tov,opp_pf,opp_pts
1,240.0,37.0,76.0,0.487,8.0,24.0,0.333,27.0,35.0,0.771,...,0.792,7.0,35.0,42.0,31.0,7.0,2.0,20.0,22.0,118.0
2,240.0,36.0,77.0,0.468,10.0,23.0,0.435,20.0,31.0,0.645,...,0.571,10.0,32.0,42.0,15.0,5.0,6.0,12.0,25.0,95.0
3,240.0,40.0,90.0,0.444,9.0,22.0,0.409,14.0,26.0,0.538,...,0.833,6.0,47.0,53.0,27.0,5.0,7.0,13.0,24.0,105.0
4,240.0,39.0,81.0,0.481,8.0,22.0,0.364,19.0,26.0,0.731,...,0.952,10.0,29.0,39.0,22.0,10.0,4.0,18.0,26.0,100.0
5,240.0,43.0,94.0,0.457,8.0,26.0,0.308,13.0,19.0,0.684,...,0.647,10.0,35.0,45.0,28.0,4.0,8.0,16.0,18.0,109.0
6,240.0,43.0,90.0,0.478,6.0,19.0,0.316,12.0,17.0,0.706,...,0.556,9.0,27.0,36.0,22.0,9.0,2.0,17.0,24.0,94.0
7,240.0,35.0,76.0,0.461,10.0,27.0,0.370,23.0,28.0,0.821,...,0.556,9.0,27.0,36.0,22.0,9.0,2.0,17.0,24.0,94.0
8,240.0,35.0,77.0,0.455,5.0,24.0,0.208,16.0,21.0,0.762,...,0.692,8.0,29.0,37.0,24.0,11.0,3.0,3.0,19.0,95.0
9,240.0,39.0,81.0,0.481,10.0,23.0,0.435,25.0,31.0,0.806,...,0.783,9.0,30.0,39.0,21.0,6.0,4.0,16.0,19.0,103.0
10,240.0,44.0,78.0,0.564,9.0,22.0,0.409,13.0,17.0,0.765,...,0.750,19.0,29.0,48.0,19.0,6.0,3.0,14.0,20.0,90.0


In [154]:
index[-2] = 83
index[-1] = 84

In [155]:
new_k.index = index

In [160]:
new_k.applymap(lambda x: round(x,3))

,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,...,opp_ft_pct,opp_orb,opp_drb,opp_trb,opp_ast,opp_stl,opp_blk,opp_tov,opp_pf,opp_pts
1,240.000,37.000,76.000,0.487,8.000,24.000,0.333,27.000,35.000,0.771,...,0.792,7.000,35.000,42.000,31.000,7.000,2.000,20.000,22.000,118.000
2,240.000,36.000,77.000,0.468,10.000,23.000,0.435,20.000,31.000,0.645,...,0.571,10.000,32.000,42.000,15.000,5.000,6.000,12.000,25.000,95.000
3,240.000,40.000,90.000,0.444,9.000,22.000,0.409,14.000,26.000,0.538,...,0.833,6.000,47.000,53.000,27.000,5.000,7.000,13.000,24.000,105.000
4,240.000,39.000,81.000,0.481,8.000,22.000,0.364,19.000,26.000,0.731,...,0.952,10.000,29.000,39.000,22.000,10.000,4.000,18.000,26.000,100.000
5,240.000,43.000,94.000,0.457,8.000,26.000,0.308,13.000,19.000,0.684,...,0.647,10.000,35.000,45.000,28.000,4.000,8.000,16.000,18.000,109.000
6,240.000,43.000,90.000,0.478,6.000,19.000,0.316,12.000,17.000,0.706,...,0.556,9.000,27.000,36.000,22.000,9.000,2.000,17.000,24.000,94.000
7,240.000,35.000,76.000,0.461,10.000,27.000,0.370,23.000,28.000,0.821,...,0.556,9.000,27.000,36.000,22.000,9.000,2.000,17.000,24.000,94.000
8,240.000,35.000,77.000,0.455,5.000,24.000,0.208,16.000,21.000,0.762,...,0.692,8.000,29.000,37.000,24.000,11.000,3.000,3.000,19.000,95.000
9,240.000,39.000,81.000,0.481,10.000,23.000,0.435,25.000,31.000,0.806,...,0.783,9.000,30.000,39.000,21.000,6.000,4.000,16.000,19.000,103.000
10,240.000,44.000,78.000,0.564,9.000,22.000,0.409,13.000,17.000,0.765,...,0.750,19.000,29.000,48.000,19.000,6.000,3.000,14.000,20.000,90.000


In [161]:
new_k

,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,fta,ft_pct,...,opp_ft_pct,opp_orb,opp_drb,opp_trb,opp_ast,opp_stl,opp_blk,opp_tov,opp_pf,opp_pts
1,240.000000,37.000000,76.000000,0.487000,8.000000,24.000000,0.333000,27.000000,35.000000,0.771000,...,0.792000,7.000000,35.000000,42.000000,31.000000,7.000000,2.000000,20.000000,22.000000,118.000000
2,240.000000,36.000000,77.000000,0.468000,10.000000,23.000000,0.435000,20.000000,31.000000,0.645000,...,0.571000,10.000000,32.000000,42.000000,15.000000,5.000000,6.000000,12.000000,25.000000,95.000000
3,240.000000,40.000000,90.000000,0.444000,9.000000,22.000000,0.409000,14.000000,26.000000,0.538000,...,0.833000,6.000000,47.000000,53.000000,27.000000,5.000000,7.000000,13.000000,24.000000,105.000000
4,240.000000,39.000000,81.000000,0.481000,8.000000,22.000000,0.364000,19.000000,26.000000,0.731000,...,0.952000,10.000000,29.000000,39.000000,22.000000,10.000000,4.000000,18.000000,26.000000,100.000000
5,240.000000,43.000000,94.000000,0.457000,8.000000,26.000000,0.308000,13.000000,19.000000,0.684000,...,0.647000,10.000000,35.000000,45.000000,28.000000,4.000000,8.000000,16.000000,18.000000,109.000000
6,240.000000,43.000000,90.000000,0.478000,6.000000,19.000000,0.316000,12.000000,17.000000,0.706000,...,0.556000,9.000000,27.000000,36.000000,22.000000,9.000000,2.000000,17.000000,24.000000,94.000000
7,240.000000,35.000000,76.000000,0.461000,10.000000,27.000000,0.370000,23.000000,28.000000,0.821000,...,0.556000,9.000000,27.000000,36.000000,22.000000,9.000000,2.000000,17.000000,24.000000,94.000000
8,240.000000,35.000000,77.000000,0.455000,5.000000,24.000000,0.208000,16.000000,21.000000,0.762000,...,0.692000,8.000000,29.000000,37.000000,24.000000,11.000000,3.000000,3.000000,19.000000,95.000000
9,240.000000,39.000000,81.000000,0.481000,10.000000,23.000000,0.435000,25.000000,31.000000,0.806000,...,0.783000,9.000000,30.000000,39.000000,21.000000,6.000000,4.000000,16.000000,19.000000,103.000000
10,240.000000,44.000000,78.000000,0.564000,9.000000,22.000000,0.409000,13.000000,17.000000,0.765000,...,0.750000,19.000000,29.000000,48.000000,19.000000,6.000000,3.000000,14.000000,20.000000,90.000000


In [184]:
d

{'ATL':    team  year     mp    fg    fga  fg_pct   fg3  fg3a  fg3_pct    ft   ...     \
 1   ATL  2014  240.0  37.0   76.0   0.487   8.0  24.0    0.333  27.0   ...      
 2   ATL  2014  240.0  36.0   77.0   0.468  10.0  23.0    0.435  20.0   ...      
 3   ATL  2014  240.0  40.0   90.0   0.444   9.0  22.0    0.409  14.0   ...      
 4   ATL  2014  240.0  39.0   81.0   0.481   8.0  22.0    0.364  19.0   ...      
 5   ATL  2014  240.0  43.0   94.0   0.457   8.0  26.0    0.308  13.0   ...      
 6   ATL  2014  240.0  43.0   90.0   0.478   6.0  19.0    0.316  12.0   ...      
 7   ATL  2014  240.0  35.0   76.0   0.461  10.0  27.0    0.370  23.0   ...      
 8   ATL  2014  240.0  35.0   77.0   0.455   5.0  24.0    0.208  16.0   ...      
 9   ATL  2014  240.0  39.0   81.0   0.481  10.0  23.0    0.435  25.0   ...      
 10  ATL  2014  240.0  44.0   78.0   0.564   9.0  22.0    0.409  13.0   ...      
 11  ATL  2014  240.0  33.0   74.0   0.446  11.0  24.0    0.458  11.0   ...      
 12  ATL 

In [310]:
big_dic['CHO']

,team,year,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,opp_ft_pct,opp_orb,opp_drb,opp_trb,opp_ast,opp_stl,opp_blk,opp_tov,opp_pf,opp_pts
3178,CHO,2015,265.0,41.0,101.0,0.406,6.0,21.0,0.286,20.0,...,0.875,3.0,32.0,35.0,25.0,9.0,10.0,14.0,25.0,106.0
3179,CHO,2015,240.0,26.0,72.0,0.361,4.0,14.0,0.286,13.0,...,0.769,9.0,36.0,45.0,16.0,8.0,0.0,18.0,22.0,71.0
3180,CHO,2015,240.0,33.0,70.0,0.471,6.0,15.0,0.400,21.0,...,0.588,8.0,27.0,35.0,23.0,5.0,0.0,12.0,26.0,96.0
3181,CHO,2015,240.0,37.0,80.0,0.463,6.0,21.0,0.286,11.0,...,1.000,11.0,36.0,47.0,26.0,5.0,6.0,12.0,19.0,100.0
3182,CHO,2015,240.0,35.0,82.0,0.427,5.0,18.0,0.278,21.0,...,0.806,6.0,30.0,36.0,19.0,6.0,4.0,8.0,25.0,89.0
3183,CHO,2015,290.0,48.0,97.0,0.495,6.0,21.0,0.286,20.0,...,0.769,7.0,31.0,38.0,28.0,8.0,3.0,19.0,33.0,119.0
3184,CHO,2015,240.0,35.0,79.0,0.443,3.0,14.0,0.214,19.0,...,0.773,8.0,30.0,38.0,28.0,8.0,2.0,11.0,23.0,107.0
3185,CHO,2015,240.0,35.0,75.0,0.467,7.0,16.0,0.438,23.0,...,0.875,15.0,33.0,48.0,25.0,6.0,4.0,12.0,21.0,102.0
3186,CHO,2015,240.0,39.0,91.0,0.429,7.0,16.0,0.438,18.0,...,0.929,11.0,34.0,45.0,16.0,3.0,6.0,11.0,23.0,95.0
3187,CHO,2015,240.0,32.0,84.0,0.381,7.0,17.0,0.412,16.0,...,0.786,11.0,40.0,51.0,30.0,7.0,3.0,14.0,25.0,112.0


In [296]:
work = dict(zip(big_dic.keys(),[big_dic[key][big_dic[key]['year']=='2014'].drop(columns=['year','team']) for key in big_dic.keys()]))
work2 = copy.deepcopy(work)

In [297]:
#work = [{k:v.drop(columns=['team','year'])} for k,v in work.items()] 

In [298]:
for k,v in work.items():
    work[k] = (v.mean(),v.std())

In [299]:
work

{'ATL': (mp             241.829268
  fg              37.329268
  fga             81.560976
  fg_pct           0.458524
  fg3              9.365854
  fg3a            25.804878
  fg3_pct          0.359793
  ft              16.975610
  fta             21.731707
  ft_pct           0.782963
  orb              8.695122
  drb             31.280488
  trb             39.975610
  ast             24.890244
  stl              8.292683
  blk              3.975610
  tov             14.500000
  pf              19.231707
  pts            101.000000
  opp_mp         242.134146
  opp_fg          38.731707
  opp_fga         83.231707
  opp_fg_pct       0.466293
  opp_fg3          7.975610
  opp_fg3a        21.853659
  opp_fg3_pct      0.361610
  opp_ft          16.134146
  opp_fta         21.743902
  opp_ft_pct       0.739512
  opp_orb         10.682927
  opp_drb         32.780488
  opp_trb         43.463415
  opp_ast         23.085366
  opp_stl          8.353659
  opp_blk          4.426829
  opp_tov    

In [300]:
work_df = pd.DataFrame(work).T

In [294]:
work_df

,0,1
ATL,mp 241.829268 fg 37.3...,mp 7.638119 fg 4.993...
BOS,mp 240.609756 fg 36.5...,mp 3.880168 fg 4.667...
BRK,mp 242.439024 fg 35.7...,mp 8.434347 fg 4.983...
CHI,mp 243.048780 fg 34.6...,mp 12.662672 fg 5.185...
CHO,mp NaN fg NaN fga ...,mp NaN fg NaN fga ...
CLE,mp 243.048780 fg 37.0...,mp 9.930502 fg 5.149...
DAL,mp 241.829268 fg 39.6...,mp 6.550478 fg 5.076...
DEN,mp 240.914634 fg 38.3...,mp 4.722421 fg 5.614...
DET,mp 241.219512 fg 38.8...,mp 5.418360 fg 4.116...
GSW,mp 241.829268 fg 39.4...,mp 6.550478 fg 5.188...


In [309]:
boxscore[boxscore.team=='CHO']

,team,year,mp,fg,fga,fg_pct,fg3,fg3a,fg3_pct,ft,...,opp_ft_pct,opp_orb,opp_drb,opp_trb,opp_ast,opp_stl,opp_blk,opp_tov,opp_pf,opp_pts
3178,CHO,2015,265.0,41.0,101.0,0.406,6.0,21.0,0.286,20.0,...,0.875,3.0,32.0,35.0,25.0,9.0,10.0,14.0,25.0,106.0
3179,CHO,2015,240.0,26.0,72.0,0.361,4.0,14.0,0.286,13.0,...,0.769,9.0,36.0,45.0,16.0,8.0,0.0,18.0,22.0,71.0
3180,CHO,2015,240.0,33.0,70.0,0.471,6.0,15.0,0.400,21.0,...,0.588,8.0,27.0,35.0,23.0,5.0,0.0,12.0,26.0,96.0
3181,CHO,2015,240.0,37.0,80.0,0.463,6.0,21.0,0.286,11.0,...,1.000,11.0,36.0,47.0,26.0,5.0,6.0,12.0,19.0,100.0
3182,CHO,2015,240.0,35.0,82.0,0.427,5.0,18.0,0.278,21.0,...,0.806,6.0,30.0,36.0,19.0,6.0,4.0,8.0,25.0,89.0
3183,CHO,2015,290.0,48.0,97.0,0.495,6.0,21.0,0.286,20.0,...,0.769,7.0,31.0,38.0,28.0,8.0,3.0,19.0,33.0,119.0
3184,CHO,2015,240.0,35.0,79.0,0.443,3.0,14.0,0.214,19.0,...,0.773,8.0,30.0,38.0,28.0,8.0,2.0,11.0,23.0,107.0
3185,CHO,2015,240.0,35.0,75.0,0.467,7.0,16.0,0.438,23.0,...,0.875,15.0,33.0,48.0,25.0,6.0,4.0,12.0,21.0,102.0
3186,CHO,2015,240.0,39.0,91.0,0.429,7.0,16.0,0.438,18.0,...,0.929,11.0,34.0,45.0,16.0,3.0,6.0,11.0,23.0,95.0
3187,CHO,2015,240.0,32.0,84.0,0.381,7.0,17.0,0.412,16.0,...,0.786,11.0,40.0,51.0,30.0,7.0,3.0,14.0,25.0,112.0


In [ ]:
#missing charlotte data from 2008-2014 because the name was CHA instead of CHO
#priority 1
#CHO is in boxscores but not in boxscore united